# SQL injection vulnerability detector

A SQL injection is a serious vulnerability affecting applications that use SQL as their database language. Through cleverly constructed text inputs that modify the backend SQL query, the attacker can send malicious SQL commands to the database server. The most common goal is bulk extraction of knowledge. <br>
SQL injection is carried out through user input. In order to scan for this vulnerability We can extract the web forms and check whether a web page has SQL errors in it.

In [32]:
#importing dependencies
import requests
from bs4 import BeautifulSoup as bs
import sys
from urllib.parse import urljoin
from pprint import pprint

In [2]:
# initialize an HTTP session & set the browser
s = requests.Session()
s.headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"

## Functions

<ol>
<li> The function get_all_forms() uses BeautifulSoup library to extract all form tags from HTML and returns them as a Python list. 
<li> get_form_details() function gets a single form tag object as an argument and parses useful information about the form, such as action (the target URL), method (GET, POST, etc) and all input field attributes (type, name and value).
</ol>    

In [33]:
def get_all_forms(url):
    '''When given a URL, this function will make a request to a page and will extract and return 
    all the HTML form tags from it.
    '''
    soup = bs(s.get(url).content, "html.parser")
    return soup.find_all("form")

In [34]:
get_all_forms("https://www.geeksforgeeks.org/python-programming-language/")

[<form class="p-relative" data-sm="false" id="gcse-form"><input aria-expanded="false" class="gcse-search-input__wrapper" id="gcse-search-input"/>
 <i class="gfg-icon gfg-icon_times gfg-icon_white hide-search"></i>
 <button class="gcse-search__btn not-expanded">
 <i class="gfg-icon gfg-icon_search gfg-icon_white gcse-search__icon"></i></button></form>]

In [36]:
form = get_all_forms("https://www.geeksforgeeks.org/python-programming-language/")
print(type(form))

<class 'bs4.element.ResultSet'>


In [35]:
def get_form_details(form):
    """
    This function extracts all possible useful information about an HTML `form`
    Like the list of tags for example
    """
    # initialize empty dictionary
    details = {}
    # get the target url and store in a variable action (form action)
    try:
        action = form.attrs.get("action").lower()
    except:
        action = None
    # get the form method (POST, GET, etc.)
    method = form.attrs.get("method", "get").lower()
    # get all the input details such as type and name
    inputs = []
    for input_tag in form.find_all("input"):
        input_type = input_tag.attrs.get("type", "text")
        input_name = input_tag.attrs.get("name")
        input_value = input_tag.attrs.get("value", "")
        inputs.append({"type": input_type, "name": input_name, "value": input_value})
    # put everything to the resulting dictionary
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    return details

In [37]:
def is_vulnerable(response):
    """This function takes the response of a page as input and returns whether a page 
    is vulnerable to SQL injection and returns a boolean accordingly."""
    errors = {
        # MySQL
        "you have an error in your sql syntax;",
        "warning: mysql",
        # SQL Server
        "unclosed quotation mark after the character string",
        # Oracle
        "quoted string not properly terminated",
    }
    for error in errors:
        # if you find one of these errors, return True
        if error in response.content.decode().lower():
            return True
    # if no error is detected
    return False

If the page has any syntax error, it is vulnerable. Despite the fact that there are a lot of database errors we will limit the search to Oracle and SQL Server Errors, because these two are mostly used.

In [44]:
def scan_sql_injection(url):
    # test on URL
    for c in "\"'":
        # add quote/double quote character to the URL
        new_url = f"{url}{c}"
        print("[!] Trying", new_url)
        # make the HTTP request
        res = s.get(new_url)
        if is_vulnerable(res):
            # SQL Injection detected on the URL itself, 
            # no need to preceed for extracting forms and submitting them
            print("[+] SQL Injection vulnerability detected, link:", new_url)
            return
    # test on HTML forms
    forms = get_all_forms(url)
    print(f"[+] Detected {len(forms)} forms on {url}.")
    for form in forms:
        form_details = get_form_details(form)
        for c in "\"'":
            # the data body we want to submit
            data = {}
            for input_tag in form_details["inputs"]:
                if input_tag["value"] or input_tag["type"] == "hidden":
                    # any input form that has some value or hidden,
                    # just use it in the form body
                    try:
                        data[input_tag["name"]] = input_tag["value"] + c
                    except:
                        pass
                elif input_tag["type"] != "submit":
                    # all others except submit, use some junk data with special character
                    data[input_tag["name"]] = f"test{c}"
            # join the url with the action (form request URL)
            url = urljoin(url, form_details["action"])
            if form_details["method"] == "post":
                res = s.post(url, data=data)
            elif form_details["method"] == "get":
                res = s.get(url, params=data)
            # test whether the resulting page is vulnerable
            if is_vulnerable(res):
                print("[+] SQL Injection vulnerability detected, link:", url)
                print("[+] Form:")
                pprint(form_details)
            else:
                print("[+] No vulnerability detected")
                break   

Testing on a site that is known to be vulnerable:
url = "http://testphp.vulnweb.com/artists.php?artist=1"

In [45]:
url = "http://testphp.vulnweb.com/artists.php?artist=1"
scan_sql_injection(url)

[!] Trying http://testphp.vulnweb.com/artists.php?artist=1"
[+] SQL Injection vulnerability detected, link: http://testphp.vulnweb.com/artists.php?artist=1"


Using the function on another page.

In [46]:
url_arg = "https://www.datasciencetech.institute/"
scan_sql_injection(url_arg)

[!] Trying https://www.datasciencetech.institute/"
[!] Trying https://www.datasciencetech.institute/'
[+] Detected 1 forms on https://www.datasciencetech.institute/.
[+] No vulnerability detected
